In [ ]:
import pandas as pd

# Import/Export Data

In [ ]:
df_csv = pd.read_csv('../data/my_csv.csv')
df_txt = pd.read_table('../data/my_table.txt')
df_excel = pd.read_excel('../data/my_excel.xlsx', 
                         header = None,
                         parse_dates = ['col2'],
                         index_col = ['col1'],
                        nrows = 2)

df_excel.to_excel('.xlsx', index = False)

# Commonly Used Methods

In [ ]:
#objects
df.values
df.index
df.dtypes
df.columns
df.shape
df.T

#descriptive
df.head(5)
df.tail(5)
df.info()
df.describe()

#stats
df.mean()
df.max()
df.count()

#unique
df['col'].unique()
df['col'].value_counts()
df.drop_duplicates(['Gender', 'Transfer'], keep='last')

In [ ]:
#Replace
df['Gender'].replace({'Female':0, 'Male':1}).head()

In [ ]:
#sort
df.sort_values(['Weight','Height'],ascending=[True,False]).head()

df.Grade = df.Grade.astype('category')
df.Grade = df.Grade.cat.reorder_categories(['Freshman', 'Sophomore', 'Junior', 'Senior'],
                                           ordered=True)

In [ ]:
#apply
df.apply(lambda x: x.mean())

# Indexing

In [ ]:
df['col1']
df.col1
df[['col1','col2']]
df.loc[~df.Weight != 70,'col1']
df.loc[df.Grade.isin(['Freshman', 'Senior'])]

condition_1 = df.School == 'Fudan University'
condition_1 = df.Grade == 'Senior'
df.loc[condition_1 | condition_2]

In [ ]:
df.query('Weight > Weight.mean()')
df.query('Grade == ["Junior", "Senior"]')
low, high =70, 80
df.query('(Weight >= @low) & (Weight <= @high)')

In [ ]:
###index
df.rename(index=lambda x:str.upper(x))
df.reset_index()

# Group

In [ ]:
df.groupby(['School', 'Gender'])['Height'].mean()

condition = df.Weight > df.Weight.mean()
df.groupby(condition)['Height'].mean()

In [ ]:
gb = df.groupby('Gender')[['Height', 'Weight']]
gb.agg({'Height':['mean','max'], 'Weight':'count'})

gb.filter(lambda x: x.shape[0] > 100).head()

def my_func(s):
    res = 'High'
    if s.mean() <= df[s.name].mean():
        res = 'Low'
    return res
gb.agg(my_func)

def BMI(x):
    Height = x['Height']/100
    Weight = x['Weight']
    BMI_value = Weight/Height**2
    return BMI_value.mean()
gb.apply(BMI)

# Transform

In [ ]:
df_melted = df.melt(id_vars = ['Class', 'Name'],
                    value_vars = ['Chinese', 'Math'],
                    var_name = 'Subject',
                    value_name = 'Grade')

df.unstack(0)

# Connect two dataframes

In [ ]:
df1.merge(df2, on = 'col1', how = 'left')
df1.join(df2, how = 'left') #merge on index
pd.concat([df1,df2], axis=0 ) #1 means horizontal

df1.append(s,ignore_index = True)
df1.assign(Grade = [1,2,3])
df1['Grade'] = [1,2,3]

# Handle Missing data

In [ ]:
#find na
df.isna()
subset = df['col1','col2']
df[sub_set.isna.any(1)].head()
#delete na
df.dropna()
#fill na
df.fillna(method = 'ffill')
df.fillna(df.mean())
df.interpolate()
df.interpolate(method = 'index')

# String Str
https://docs.python.org/3/library/re.html

In [ ]:
df['col'].astype('string')
df['col'].str.split('[abc]')
df['col'].str.join('-')
df['col'].str.contains(',')
df['col'].str.find('apple')
df['col'].str.replace(',','',regex=True)
pat = '(?P<市名>\w+市)(?P<区名>\w+区)(?P<路名>\w+路)(?P<编号>\d+号)'

df.str.extract(pat)
df.str.upper()
df.str.title()
df.str.capitalize()
df.str.swapcase()
df.str.len()

df.str.strip()
df.str.pad(5,'left','*')
df.str.zfill(6)

# Method chaining

In [ ]:
def read(fp):
    df = (pd.read_csv(fp)
            .rename(columns=str.lower)
            .drop('unnamed: 36', axis=1)
            .pipe(extract_city_name)
            .pipe(time_to_datetime, ['dep_time', 'arr_time', 'crs_arr_time', 'crs_dep_time'])
            .assign(fl_date=lambda x: pd.to_datetime(x['fl_date']),
                    dest=lambda x: pd.Categorical(x['dest']),
                    origin=lambda x: pd.Categorical(x['origin']),
                    tail_num=lambda x: pd.Categorical(x['tail_num']),
                    unique_carrier=lambda x: pd.Categorical(x['unique_carrier']),
                    cancellation_code=lambda x: pd.Categorical(x['cancellation_code'])))
    return df

def extract_city_name(df):
    '''
    Chicago, IL -> Chicago for origin_city_name and dest_city_name
    '''
    cols = ['origin_city_name', 'dest_city_name']
    city = df[cols].apply(lambda x: x.str.extract("(.*), \w{2}", expand=False))
    df = df.copy()
    df[['origin_city_name', 'dest_city_name']] = city
    return df

def time_to_datetime(df, columns):
    '''
    Combine all time items into datetimes.

    2014-01-01,0914 -> 2014-01-01 09:14:00
    '''
    df = df.copy()
    def converter(col):
        timepart = (col.astype(str)
                       .str.replace('\.0$', '')  # NaNs force float dtype
                       .str.pad(4, fillchar='0'))
        return pd.to_datetime(df['fl_date'] + ' ' +
                               timepart.str.slice(0, 2) + ':' +
                               timepart.str.slice(2, 4),
                               errors='coerce')
    df[columns] = df[columns].apply(converter)
    return df

In [ ]:
(df.dropna(subset=['dep_time', 'unique_carrier'])
   .loc[df['unique_carrier']
       .isin(df['unique_carrier'].value_counts().index[:5])]
   .set_index('dep_time')
   # TimeGrouper to resample & groupby at once
   .groupby(['unique_carrier', pd.TimeGrouper("H")])
   .fl_num.count()
   .unstack(0)
   .fillna(0)
   .rolling(24)
   .sum()
   .rename_axis("Flights per Day", axis=1)
   .plot()
)
sns.despine()

In [ ]:
plt.figure(figsize=(15, 5))
(df[['fl_date', 'tail_num', 'dep_time', 'dep_delay']]
    .dropna()
    .assign(hour=lambda x: x.dep_time.dt.hour)
    .query('5 < dep_delay < 600')
    .pipe((sns.boxplot, 'data'), 'hour', 'dep_delay'))
sns.despine()

In [ ]:
def mode(x):
    '''
    Arbitrarily break ties.
    '''
    return x.value_counts().index[0]

aggfuncs = {'tmpf': 'mean', 'relh': 'mean',
            'sped': 'mean', 'mslp': 'mean',
            'p01i': 'mean', 'vsby': 'mean',
            'gust_mph': 'mean', 'skyc1': mode,
            'skyc2': mode, 'skyc3': mode}
# TimeGrouper works on a DatetimeIndex, so we move `station` to the
# columns and then groupby it as well.
daily = (weather.reset_index(level="station")
                .groupby([pd.TimeGrouper('1d'), "station"])
                .agg(aggfuncs))

daily.head()